In [36]:
#!pip install grad-cam

In [38]:
'''
from google.colab import files

uploaded = files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d amanrawat001/celeb-df-preprocessed

with zipfile.ZipFile("/content/celeb-df-preprocessed.zip", 'r') as zip_ref:
  zip_ref.extractall()
'''

'\nfrom google.colab import files\n\nuploaded = files.upload()\n\n!mkdir ~/.kaggle\n!cp kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n\n!kaggle datasets download -d amanrawat001/celeb-df-preprocessed\n\nwith zipfile.ZipFile("/content/celeb-df-preprocessed.zip", \'r\') as zip_ref:\n  zip_ref.extractall()\n'

In [40]:
df_train = pd.read_csv("/kaggle/input/deepfake-metadata/train_metadata.csv")
df_val = pd.read_csv("/kaggle/input/deepfake-metadata/val_metadata.csv")
df_test = pd.read_csv("/kaggle/input/deepfake-metadata/test_metadata.csv")

print(f"Train: {len(df_train)} videos")
print(f"Val: {len(df_val)} videos")
print(f"Test: {len(df_test)} videos")

Train: 2758 videos
Val: 597 videos
Test: 587 videos


In [41]:
df_train.head(3)

,label,type,path,video_folder,frames,source_id
0,real,real,/kaggle/input/faceforencispp-extracted-frames/...,437,32,437
1,real,real,/kaggle/input/faceforencispp-extracted-frames/...,515,32,515
2,real,real,/kaggle/input/faceforencispp-extracted-frames/...,248,32,248


In [42]:
df_val.head(3)

,label,type,path,video_folder,frames,source_id
0,real,real,/kaggle/input/faceforencispp-extracted-frames/...,973,32,973
1,real,real,/kaggle/input/faceforencispp-extracted-frames/...,768,32,768
2,real,real,/kaggle/input/faceforencispp-extracted-frames/...,057,32,57


In [43]:
df_test.tail(3)

,label,type,path,video_folder,frames,source_id
584,fake,FaceSwap,/kaggle/input/faceforencispp-extracted-frames/...,179_212,32,179
585,fake,FaceSwap,/kaggle/input/faceforencispp-extracted-frames/...,646_643,32,646
586,fake,FaceSwap,/kaggle/input/faceforencispp-extracted-frames/...,886_877,32,886


In [44]:
df_train.iloc[0]

label                                                        real
type                                                         real
path            /kaggle/input/faceforencispp-extracted-frames/...
video_folder                                                  437
frames                                                         32
source_id                                                     437
Name: 0, dtype: object

In [45]:
df_train['path'].iloc[1000]

'/kaggle/input/faceforencispp-extracted-frames/fake/Face2Face/605_591'

In [58]:
import torch
import torch.nn as nn
import timm
import pickle
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import GradScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image
import os
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

df_train = pd.read_csv("/kaggle/input/deepfake-metadata/train_metadata.csv")
df_val = pd.read_csv("/kaggle/input/deepfake-metadata/val_metadata.csv")
df_test = pd.read_csv("/kaggle/input/deepfake-metadata/test_metadata.csv")

print(f"Train: {len(df_train)} videos")
print(f"Val: {len(df_val)} videos")
print(f"Test: {len(df_test)} videos")

print("\nClass distribution:")
print(df_train['label'].value_counts())
print(df_train['label'].value_counts(normalize=True))

with open('/kaggle/input/deepfake-metadata/classical_features.pkl', 'rb') as f:
    classical_data = pickle.load(f)

train_classical = classical_data['train_features']
val_classical = classical_data['val_features']
test_classical = classical_data['test_features']

print(f"\nClassical features shape: Train={train_classical.shape}, Val={val_classical.shape}, Test={test_classical.shape}")

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_classical = scaler.fit_transform(train_classical)
val_classical = scaler.transform(val_classical)
test_classical = scaler.transform(test_classical)

classical_dim = train_classical.shape[1]
print(f"Classical feature dimension: {classical_dim}")

train_transform = A.Compose([
    A.ImageCompression(quality_lower=30, quality_upper=100, p=0.7),
    A.OneOf([
        A.Downscale(scale_min=0.6, scale_max=0.9, p=1),
        A.Resize(180, 180, p=1),
    ], p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=1),
        A.MotionBlur(blur_limit=5, p=1),
        A.GaussianBlur(blur_limit=(3, 7), p=1.0)
    ], p=0.3),
    A.OneOf([
        A.RandomBrightnessContrast(0.15, 0.15, p=1),
        A.HueSaturationValue(10, 15, 15, p=1),
    ], p=0.3),
    A.HorizontalFlip(p=0.5),
    A.Affine(shift={'x': (0, 0.05), 'y': (0, 0.05)}, scale=(0.9, 1.1), 
             rotate=(-5, 5), p=0.3),
    A.Resize(224, 224),
    A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

class VideoDatasetWithClassical(Dataset):
    def __init__(self, dataframe, classical_features, transform=None, frames_per_video=16):
        self.df = dataframe.reset_index(drop=True)
        self.classical_features = classical_features
        self.transform = transform
        self.frames_per_video = frames_per_video
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = row['path']
        label = 1 if row['label'] == 'fake' else 0
        
        frame_files = sorted(os.listdir(video_path))
        total_frames = len(frame_files)
        
        if total_frames == 0:
            raise ValueError(f"No frames found in {video_path}")
        
        if total_frames <= self.frames_per_video:
            indices = list(range(total_frames))
        else:
            step = total_frames / self.frames_per_video
            indices = [int(i * step) for i in range(self.frames_per_video)]
        
        frames = []
        for i in indices:
            img_path = os.path.join(video_path, frame_files[i])
            img = Image.open(img_path).convert('RGB')
            img = np.array(img)
            
            if self.transform:
                augmented = self.transform(image=img)
                img = augmented['image']
            
            frames.append(img)
        
        while len(frames) < self.frames_per_video:
            frames.append(frames[-1])
        
        frames_tensor = torch.stack(frames)
        classical_feat = torch.tensor(self.classical_features[idx], dtype=torch.float32)
        
        return frames_tensor, classical_feat, label

train_dataset = VideoDatasetWithClassical(df_train, train_classical, train_transform, 16)
val_dataset = VideoDatasetWithClassical(df_val, val_classical, val_transform, 16)
test_dataset = VideoDatasetWithClassical(df_test, test_classical, val_transform, 16)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

print("Dataloaders created with classical features")

class FusionModel(nn.Module):
    def __init__(self, model_name='efficientnet_b0', classical_dim=72, num_classes=2, 
                 hidden_size=128, num_layers=1, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            features = self.backbone(dummy)
            self.feature_dim = features.shape[1]
        
        self.lstm = nn.LSTM(
            input_size=self.feature_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0
        )
        
        self.classical_proj = nn.Sequential(
            nn.Linear(classical_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        fusion_dim = hidden_size + 32
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(fusion_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, video, classical_feat):
        batch_size, num_frames, c, h, w = video.shape
        
        video_flat = video.view(batch_size * num_frames, c, h, w)
        frame_features = self.backbone(video_flat)
        frame_features = frame_features.view(batch_size, num_frames, -1)
        
        lstm_out, _ = self.lstm(frame_features)
        lstm_features = lstm_out[:, -1, :]
        
        classical_features = self.classical_proj(classical_feat)
        
        fused = torch.cat([lstm_features, classical_features], dim=1)
        
        output = self.classifier(fused)
        
        return output

def train_one_epoch(model, train_loader, optimizer, loss_fn, device, scaler):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    for batch_idx, (videos, classical, labels) in enumerate(tqdm(train_loader, desc="Training")):
        videos, classical, labels = videos.to(device), classical.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(videos, classical)
            loss = loss_fn(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.detach().cpu().numpy())
        
        if batch_idx % 10 == 0:
            torch.cuda.empty_cache()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="binary")
    precision = precision_score(all_labels, all_preds, average="binary", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="binary", zero_division=0)
    auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else 0
    
    return avg_loss, accuracy, f1, precision, recall, auc

def validate(model, val_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for videos, classical, labels in tqdm(val_loader, desc="Validation"):
            videos, classical, labels = videos.to(device), classical.to(device), labels.to(device)
            
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(videos, classical)
                loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_loss = total_loss / len(val_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="binary", zero_division=0)
    precision = precision_score(all_labels, all_preds, average="binary", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="binary", zero_division=0)
    auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else 0
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Real', 'Fake']))
    
    return avg_loss, accuracy, f1, precision, recall, auc, all_probs, all_labels

torch.cuda.empty_cache()
gc.collect()

model = FusionModel('efficientnet_b0', classical_dim=classical_dim, num_classes=2, 
                    hidden_size=128, num_layers=1)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scaler = GradScaler()
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

def calculate_class_weights(df_train):
    num_real = len(df_train[df_train['label'] == 'real'])
    num_fake = len(df_train[df_train['label'] == 'fake'])
    total = num_real + num_fake
    weight_real = total / (2 * num_real)
    weight_fake = total / (2 * num_fake)
    return torch.tensor([weight_real, weight_fake], dtype=torch.float32)

class_weights = calculate_class_weights(df_train)
print(f"\nClass weights: Real={class_weights[0]:.4f}, Fake={class_weights[1]:.4f}")
loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))

epochs = 5
patience = 5
best_val_f1 = 0
counter = 0
save_path = "/kaggle/working/best_fusion_model.pth"

history = {
    'train_loss': [], 'train_acc': [], 'train_f1': [], 'train_auc': [],
    'val_loss': [], 'val_acc': [], 'val_f1': [], 'val_auc': []
}

print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n{'='*70}")
print(f"{'TRAINING START - FUSION MODEL':^70}")
print(f"{'='*70}\n")

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    
    train_loss, train_acc, train_f1, train_prec, train_rec, train_auc = train_one_epoch(
        model, train_loader, optimizer, loss_fn, device, scaler
    )
    
    val_loss, val_acc, val_f1, val_prec, val_rec, val_auc, _, _ = validate(
        model, val_loader, loss_fn, device
    )
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['train_auc'].append(train_auc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    history['val_auc'].append(val_auc)
    
    scheduler.step()
    
    print(f"\nTRAIN: Loss={train_loss:.4f} Acc={train_acc:.4f} F1={train_f1:.4f} AUC={train_auc:.4f}")
    print(f"VAL:   Loss={val_loss:.4f} Acc={val_acc:.4f} F1={val_f1:.4f} AUC={val_auc:.4f}")
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'val_f1': val_f1,
            'val_auc': val_auc,
        }, save_path)
        counter = 0
        print(f"Best model saved! (F1: {val_f1:.4f})")
    else:
        counter += 1
        print(f"No improvement ({counter}/{patience})")
    
    if counter >= patience:
        print("Early stopping triggered")
        break
    
    torch.cuda.empty_cache()
    gc.collect()

print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)

Device: cuda
Train: 2758 videos
Val: 597 videos
Test: 587 videos

Class distribution:
label
fake    2079
real     679
Name: count, dtype: int64
label
fake    0.753807
real    0.246193
Name: proportion, dtype: float64

Classical features shape: Train=(2758, 72), Val=(597, 72), Test=(587, 72)
Classical feature dimension: 72
Dataloaders created with classical features


/tmp/ipykernel_38/2950970756.py:54: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=100, p=0.7),
/tmp/ipykernel_38/2950970756.py:56: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(scale_min=0.6, scale_max=0.9, p=1),
/tmp/ipykernel_38/2950970756.py:60: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=1),
/tmp/ipykernel_38/2950970756.py:69: UserWarning: Argument(s) 'shift' are not valid for transform Affine
  A.Affine(shift={'x': (0, 0.05), 'y': (0, 0.05)}, scale=(0.9, 1.1),
/tmp/ipykernel_38/2950970756.py:282: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Class weights: Real=2.0309, Fake=0.6633

Model parameters: 4,765,214

                    TRAINING START - FUSION MODEL                     


Epoch 1/5


Validation: 100%|██████████| 75/75 [01:19<00:00,  1.07s/it]



Classification Report:
              precision    recall  f1-score   support

        Real       0.43      0.65      0.51       147
        Fake       0.86      0.72      0.78       450

    accuracy                           0.70       597
   macro avg       0.64      0.68      0.65       597
weighted avg       0.75      0.70      0.72       597


TRAIN: Loss=0.6823 Acc=0.7060 F1=0.8183 AUC=0.5637
VAL:   Loss=0.5533 Acc=0.6985 F1=0.7816 AUC=0.7685
Best model saved! (F1: 0.7816)

Epoch 2/5


...


Classification Report:
              precision    recall  f1-score   support

        Real       0.78      0.69      0.73       147
        Fake       0.90      0.94      0.92       450

    accuracy                           0.88       597
   macro avg       0.84      0.81      0.83       597
weighted avg       0.87      0.88      0.87       597


TRAIN: Loss=0.4861 Acc=0.8252 F1=0.8827 AUC=0.8577
VAL:   Loss=0.3363 Acc=0.8760 F1=0.9192 AUC=0.9009
Best model saved! (F1: 0.9192)

Epoch 4/5


Validation: 100%|██████████| 75/75 [00:34<00:00,  2.19it/s]



Classification Report:
              precision    recall  f1-score   support

        Real       0.54      0.97      0.69       147
        Fake       0.99      0.73      0.84       450

    accuracy                           0.79       597
   macro avg       0.76      0.85      0.77       597
weighted avg       0.88      0.79      0.80       597


TRAIN: Loss=0.4258 Acc=0.8565 F1=0.9044 AUC=0.8942
VAL:   Loss=0.5406 Acc=0.7889 F1=0.8393 AUC=0.9277
No improvement (1/5)

Epoch 5/5


Validation: 100%|██████████| 75/75 [00:35<00:00,  2.12it/s]



Classification Report:
              precision    recall  f1-score   support

        Real       0.68      0.95      0.79       147
        Fake       0.98      0.85      0.91       450

    accuracy                           0.88       597
   macro avg       0.83      0.90      0.85       597
weighted avg       0.91      0.88      0.88       597


TRAIN: Loss=0.3481 Acc=0.8964 F1=0.9305 AUC=0.9334
VAL:   Loss=0.3785 Acc=0.8777 F1=0.9132 AUC=0.9607
No improvement (2/5)

TRAINING COMPLETE
